In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Bidirectional
from tensorflow.keras.layers import Embedding, Dropout
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import precision_score, recall_score, f1_score
import os

In [ ]:
# Fix random seed for reproducibility
np.random.seed(7)
tf.random.set_seed(7)

# Load the IMDB dataset
top_words = 5000  # Vocabulary size
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

In [6]:
# Pad sequences to ensure uniform length
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [7]:
# Load GloVe embeddings
def load_glove_embeddings(glove_file_path, word_index, embedding_dim=100):
    embeddings_index = {}
    with open(glove_file_path, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    
    embedding_matrix = np.zeros((top_words, embedding_dim))
    for word, i in word_index.items():
        if i < top_words:
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
    return embedding_matrix


In [9]:
# Note: Download GloVe embeddings (e.g., glove.6B.100d.txt) and specify the path
# For this example, assume glove.6B.100d.txt is in the working directory
glove_path = 'glove.6B.100d.txt'
embedding_dim = 100
word_index = imdb.get_word_index()
embedding_matrix = load_glove_embeddings(glove_path, word_index, embedding_dim)

In [10]:
# Create the model
model = Sequential()
model.add(Embedding(top_words, embedding_dim, input_length=max_review_length,
                    weights=[embedding_matrix], trainable=False))
model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

In [13]:
# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer,
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# Model summary
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 100)          500000    
                                                                 
 conv1d (Conv1D)             (None, 496, 64)           32064     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 248, 64)          0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 248, 64)           0         
                                                                 
 bidirectional (Bidirectiona  (None, 248, 256)         197632    
 l)                                                              
                                                                 
 dropout_1 (Dropout)         (None, 248, 256)          0

 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 894,177
Trainable params: 394,177
Non-trainable params: 500,000
_________________________________________________________________
None


In [15]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                    epochs=20, batch_size=32, callbacks=[early_stopping, lr_scheduler])


Epoch 1/20
782/782 [==============================] - 863s 1s/step - loss: 0.6521 - accuracy: 0.5969 - precision: 0.6009 - recall: 0.5772 - val_loss: 0.7501 - val_accuracy: 0.5436 - val_precision: 0.5233 - val_recall: 0.9810 - lr: 0.0010
Epoch 2/20
782/782 [==============================] - 853s 1s/step - loss: 0.6017 - accuracy: 0.6652 - precision: 0.6647 - recall: 0.6666 - val_loss: 0.5626 - val_accuracy: 0.7081 - val_precision: 0.6864 - val_recall: 0.7662 - lr: 0.0010
Epoch 3/20
782/782 [==============================] - 715s 915ms/step - loss: 0.5449 - accuracy: 0.7201 - precision: 0.7178 - recall: 0.7253 - val_loss: 0.4974 - val_accuracy: 0.7557 - val_precision: 0.7486 - val_recall: 0.7698 - lr: 0.0010
Epoch 4/20
782/782 [==============================] - 721s 923ms/step - loss: 0.4890 - accuracy: 0.7599 - precision: 0.7568 - recall: 0.7659 - val_loss: 0.4612 - val_accuracy: 0.7769 - val_precision: 0.7633 - val_recall: 0.8029 - lr: 0.0010
Epoch 5/20
782/782 [======================

In [16]:
# Evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print(f"Accuracy: {scores[1]*100:.2f}%")
print(f"Precision: {scores[2]*100:.2f}%")
print(f"Recall: {scores[3]*100:.2f}%")

# Calculate F1-score
y_pred = (model.predict(X_test) > 0.5).astype("int32")
f1 = f1_score(y_test, y_pred)
print(f"F1-Score: {f1*100:.2f}%")

# Save the model
model.save('sentiment_analysis_model_improved.h5')
print("Saved model to disk")

# Optional: Load and verify the model
loaded_model = tf.keras.models.load_model('sentiment_analysis_model_improved.h5')
print("Model Loaded")

Accuracy: 80.66%
Precision: 78.35%
Recall: 84.74%
782/782 [==============================] - 248s 316ms/step
F1-Score: 81.42%
Saved model to disk
Model Loaded
